<h2>В этот блокноте собираю данные о аренде жилья в Алмате с популярного веб-сайта krisha.kz </h2>
в ходе работы использовала библиотеки:
<li> Requests - для получения данных с сайта</li>
<li> BeutifulSoup-для работы с html кодом, для поиска данных в коде</li>
<li> Pandas-для работы со строками в базе</li>


In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

Создадим пустые листы, куда будем добавлять строки для сохранения в DataFrame
<li>information - основная информация(количество комнат, квадратура, этаж)</li>
<li>cost - цена в тенге за месяц аренды</li>
<li>place - адрес (район, пересечение улиц)</li>

In [2]:

information = []
cost = []
place = []

По поиску на сайте вышло около 300 страниц. Каждая новая страница не дополняет, а создает новый html код, нужно запрашивать каждую страницу отдельно. Создадим функцию для получения интересующих нас данных. 
<br> 
Функция принимает тип BeutifulSoup, ищет карточку каждого отдельного объявления, и сохраняет/возвращет информацию, цену и адрес.

In [3]:
def scrap(soup):
    cards = soup.find_all('div', class_='a-card a-storage-live ddl_product ddl_product_link not-colored is-visible')
    info = []
    price = []
    address = []
    company = []
    for item in cards:
        header_left = item.find('div', class_='a-card__header-left')
        info.append(header_left.find('a', class_='a-card__title').get_text())
        price.append(item.find('div', class_='a-card__price').get_text(strip=True).replace(u'\xa0', u''))
        address.append(item.find('div', class_='a-card__wrapper-subtitle').get_text(strip=True))
    return info, price, address

Запишем ссылку, в которую в цикле будем добавлять номер страницы и получать код

In [4]:
url = 'https://krisha.kz/arenda/kvartiry/almaty/?das[rent.period]=2&page='
for i in range(1,323):
    url = url+str(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    info, price, address = scrap(soup)
    for item in info:
        information.append(item)
    for item in price:
        cost.append(item)
    for item in address:
        place.append(item)

    
    

Проверим равное ли количество строк в списках

In [5]:
len(information),\
len(place),\
len(cost)


(2060, 2060, 2060)

Сохраним данные в DataFrame

In [28]:
appartment = pd.DataFrame({
    "information": information,
    "cost_kzt": cost,
    "place": place,
})

Осталось только очистка и разделение данных на столбцы

In [29]:

appartment['rooms'], appartment['total_area'],appartment['floor'] = appartment['information'].str.split(',').str
appartment['district'], appartment['intersection_of_streets'] = appartment['place'].str.split(',', 1).str
appartment['app_floor'], appartment['number_of_floors'] = appartment['floor'].str.split('/').str


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  This is separate from the ipykernel package so we can avoid doing imports until


Удалим не нужные повторяюшиеся слова, которые не влияют на смысл строк

In [30]:
appartment.rooms = appartment.rooms.str.strip('-комнатная квартира').astype(int)
appartment.floor = appartment.floor.str.strip(' этаж помесячно')
appartment.cost_kzt = appartment.cost_kzt.str.strip('〒')
appartment.number_of_floors = appartment.number_of_floors.str.strip('этаж помесячно')

appartment.total_area = appartment.total_area.str.extract('(\d+)')
appartment.drop(['information', 'place', 'floor'],axis='columns', inplace=True)

In [31]:
appartment.shape

(2060, 7)

In [32]:
appartment.head()

,cost_kzt,rooms,total_area,district,intersection_of_streets,app_floor,number_of_floors
0,250000,3,76,Медеуский р-н,Тулебаева — Кабанбай Батыра,2,4
1,130000,3,71,Алатауский р-н,3-я — Момышулы,6,12
2,100000,1,40,Алмалинский р-н,Шагабутдинова — Толе би,2,9
3,150000,3,60,Алмалинский р-н,Байтурсынова — Гоголя,4,5
4,340000,3,100,Бостандыкский р-н,Каблукова 270 — проспект Аль-Фараби,5,25


Запишем все в csv документ

In [33]:
appartment.to_csv('rent_almaty.csv')